In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("../health_registry.csv")

# Basic info
print("Shape of dataset:", df.shape)
display(df.head())


In [ ]:
df.info()


In [ ]:
df.columns = [col.replace('_', ' ').title() for col in df.columns]

In [ ]:
display(df.head())

In [ ]:
df.rename(columns={"Capacity": "Bed Capacity"}, inplace=True)
# remove duplicates
df.drop_duplicates(inplace=True)
df.head(10)



In [ ]:
# Clean the Facility Id

import re

def clean_facility_id(fid):
    if pd.isna(fid):
        return None
    fid = str(fid).strip()
    fid = re.sub(r"[^\d]", "", fid)  # remove non-numeric
    if fid.isdigit():
        return f"HF-{int(fid):04d}"
    return fid  # fallback

df["Facility Id"] = df["Facility Id"].apply(clean_facility_id)


In [ ]:
df.head(10)

In [ ]:
duplicates = df[df.duplicated(subset=[
    "Facility Id", "Facility Name", "Region", "Inspection Date"
], keep=False)]

display(duplicates)


In [ ]:
#dropping Duplicates again.
df.drop_duplicates(subset=[
    "Facility Id", "Facility Name", "Region", "Inspection Date"
], keep='first', inplace=True)
display(df.head(100))


In [ ]:
import numpy as np

# Extract numeric digits from Facility Id (e.g., "HF-0003" → 3)
df['Facility Id Num'] = df['Facility Id'].str.extract(r'(\d+)').astype(float)
df.sort_values(by='Facility Id Num', inplace=True, na_position='last')
df.drop(columns='Facility Id Num', inplace=True)

#df.sort_values(by='Facility Id', ascending=True, inplace=True)
#df.sort_values(by='Facility Id', na_position='last', inplace=True)

display(df.head(10))

In [ ]:
#cleaning up facility names 
# Define abbreviation replacements used inside the cleaning function
replacements = {
    r"\bHosp\.?\b": "Hospital",
    r"\bHlth Ctr\.?\b": "Health Center",
    r"\bCtr\.?\b": "Center",
    r"\bMed\.?\b": "Medical",
    r"\bPLC\b": ""
}

def clean_and_expand_facility_name(name):
    if pd.isna(name):
        return name
    # Remove (parenthetical)
    name = re.sub(r"\s*\([^)]*\)", "", name)
    # Remove emojis and unwanted characters
    name = re.sub(r"[^\w\s.,'-]", "", name)
    # Replace common abbreviations
    for pattern, replacement in replacements.items():
        name = re.sub(pattern, replacement, name, flags=re.IGNORECASE)
    # Remove trailing punctuation like dots or spaces
    name = re.sub(r"[.,;:\s]+$", "", name)
    return name.strip()

df["Facility Name"] = df["Facility Name"].apply(clean_and_expand_facility_name)
display(df.head(10))


In [ ]:
import re

# Function to remove emojis from any string
def remove_emojis(text):
    if pd.isna(text):
        return text
    return re.sub(r"[^\w\s.,'-]", "", str(text))

# Apply to Facility Type
df["Facility Type"] = df["Facility Type"].apply(remove_emojis)

# Apply to Remarks, then fill blanks or NaNs with "-"
df["Remarks"] = df["Remarks"].apply(remove_emojis)
df["Remarks"] = df["Remarks"].fillna("-")
df["Remarks"] = df["Remarks"].replace("", "-")  # in case some are blank strings

# Map known messy labels to standardized values
facility_type_map = {
    r"\bCHC\b": "Community Health Center",
    r"Community Health Ctr\.?": "Community Health Center",
    r"Community Health Centre": "Community Health Center",
    r"Health Ctr\.?": "Health Center",
    r"Health Centre": "Health Center",
    r"Polyclinic": "Polyclinic",
    r"Hospital\.?": "Hospital",
    r"Hosp\.?": "Hospital",
    r"Clinic": "Clinic",
    r"Clnc": "Clinic",
}

def normalize_facility_type(value):
    if pd.isna(value):
        return value
    value = re.sub(r"[^\w\s.,'-]", "", str(value))  # remove emojis
    for pattern, replacement in facility_type_map.items():
        value = re.sub(pattern, replacement, value, flags=re.IGNORECASE)
    return value.strip().title()

df["Facility Type"] = df["Facility Type"].apply(normalize_facility_type)

df[["Facility Type", "Remarks"]].drop_duplicates().head(20)


In [ ]:
df.head(20)

In [ ]:
df["Facility Type"].value_counts()


In [ ]:
# Map specific incorrect or overly granular labels to final categories
final_facility_type_map = {
    "Health Center": "Community Health Center",
    "Clinic": "Polyclinic",
    "Hospitalital": "Hospital"
}

# Apply the mapping
df["Facility Type"] = df["Facility Type"].replace(final_facility_type_map)
df["Facility Type"].value_counts()
df.head(20)


In [ ]:
# cleaning the Bed Capacity column
# Map common word numbers to digits
word_to_num = {
    "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
    "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10,
    "eleven": 11, "twelve": 12, "thirteen": 13, "fourteen": 14,
    "fifteen": 15, "sixteen": 16, "seventeen": 17, "eighteen": 18,
    "nineteen": 19, "twenty": 20
}

import re

def clean_bed_capacity(value):
    if pd.isna(value):
        return None
    value = str(value).strip().lower()

    # Replace word numbers with digits
    for word, num in word_to_num.items():
        if word in value:
            return num

    # Extract numeric portion from strings like "156 bed", "488"
    match = re.search(r'\d+', value)
    return int(match.group()) if match else None

# Apply the cleaning function
df["Bed Capacity"] = df["Bed Capacity"].apply(clean_bed_capacity).astype("Int64")
df["Bed Capacity"].describe()



In [ ]:
df[df["Bed Capacity"].isna()][["Facility Name", "Bed Capacity"]]


In [ ]:
df.head(20)

In [ ]:
df.columns

In [ ]:
df["Licence Issue Date"]

In [ ]:
df.head(20)

In [ ]:
# cleaning the region column
import re
from rapidfuzz import process, fuzz

# ✅ Official Barbados parishes (target output format)
valid_parishes = [
    "Christ Church", "St. Andrew", "St. George", "St. James", "St. John",
    "St. Joseph", "St. Lucy", "St. Michael", "St. Peter", "St. Philip", "St. Thomas"
]

# 🛠️ Manual correction map (lowercased)
manual_corrections = {
    "stjoseph": "St. Joseph",
    "stjohn": "St. John",
    "stjames": "St. James",
    "stmichael": "St. Michael",
    "stlucy": "St. Lucy",
    "stgeorge": "St. George",
    "standrew": "St. Andrew",
    "stpeter": "St. Peter",
    "retep ts": "St. Peter",
    "nhoj ts": "St. John",
    "ycul ts": "St. Lucy",
    "hcruhc tsirhc": "Christ Church",
    "werdna ts": "St. Andrew",
    "egroeg ts": "St. George",
    "hpesoj ts": "St. Joseph",
    "leahcim ts": "St. Michael",
    "semaj ts": "St. James"
}

# 🧼 Main cleaning function
def clean_region(value):
    if not pd.notna(value):
        return None

    val = str(value).strip().lower()
    val = re.sub(r"[^\w\s]", "", val)         # remove punctuation
    val = val.replace("parish", "").strip()   # remove "parish" word

    # Manual fix if matched
    if val in manual_corrections:
        return manual_corrections[val]

    # Fix glued "stlucy" → "St. Lucy"
    if val.startswith("st") and len(val) > 2:
        for parish in valid_parishes:
            if parish.lower().replace(".", "").replace(" ", "") == val:
                return parish

    # Fuzzy fallback match
    result = process.extractOne(val, valid_parishes, scorer=fuzz.token_sort_ratio)
    if result and result[1] >= 85:
        return result[0]

    return None  # force null if no good match

# ✅ Apply correction
df["Region"] = df["Region"].apply(clean_region)


In [ ]:
df["Region"].value_counts()


In [ ]:
df.head(20)

In [ ]:
# 🧼 Clean + Normalize Licence Issue Date
df["Licence Issue Date"] = pd.to_datetime(df["Licence Issue Date"], errors="coerce", dayfirst=True)

# Format to 'DD-MM-YYYY' string
df["Licence Issue Date"] = df["Licence Issue Date"].dt.strftime('%d-%m-%Y')
df.head(20)

In [ ]:
# 🧼 Clean + Normalize Inspection Date
df["Inspection Date"] = pd.to_datetime(df["Inspection Date"], errors="coerce", dayfirst=True)

# Format to 'DD-MM-YYYY' string (only if date parsing succeeded)
df["Inspection Date"] = df["Inspection Date"].dt.strftime('%d-%m-%Y')


In [ ]:
df["Inspection Date"].head(10)


In [ ]:
df.head(20)

In [ ]:
import re
import numpy as np

def clean_gps(gps):
    if not pd.notna(gps):
        return np.nan

    # Remove brackets, emojis, and extra characters
    gps = re.sub(r"[^\d.,\s\-]", "", gps)
    
    # Split by comma or space
    parts = re.split(r"[,\s]+", gps.strip())
    
    # Filter out empty strings
    parts = [p for p in parts if p]

    # Expect exactly 2 parts (lat, lon)
    if len(parts) != 2:
        return np.nan

    try:
        lat = round(float(parts[0]), 5)
        lon = round(float(parts[1]), 5)
        return f"{lat}, {lon}"
    except:
        return np.nan


In [ ]:
df["Gps Location"] = df["Gps Location"].apply(clean_gps)


In [ ]:
df["Gps Location"].head(30)

In [ ]:
df.head(30)

In [ ]:
# Remove newline characters and the literal string "NewLine"
df["Facility Name"] = df["Facility Name"].str.replace(r"\n", " ", regex=True)
df["Facility Name"] = df["Facility Name"].str.replace(r"NewLine", " ", regex=True)

# Optional: remove extra spaces
df["Facility Name"] = df["Facility Name"].str.strip().replace(r"\s+", " ", regex=True)


In [ ]:
df["Facility Name"].head(10)
df["Facility Name"].str.contains("NewLine|\n", regex=True).sum()  # Should return 0


In [ ]:
df["Facility Type"] = df["Facility Type"].fillna("-")

# Also handle empty strings or whitespace-only entries
df["Facility Type"] = df["Facility Type"].replace(r"^\s*$", "-", regex=True)


In [ ]:
df["Facility Type"].value_counts(dropna=False)


In [ ]:
df["Bed Capacity"] = df["Bed Capacity"].fillna(0)
df["Bed Capacity"] = df["Bed Capacity"].replace(r"^\s*$", 0, regex=True)

# Convert to integer (or float if needed)
df["Bed Capacity"] = pd.to_numeric(df["Bed Capacity"], errors="coerce").fillna(0).astype(int)


In [ ]:
df["Bed Capacity"].value_counts(dropna=False).head()
df["Bed Capacity"].dtype


In [ ]:
# Replace actual None or NaN
df["Region"] = df["Region"].fillna("-")

# Replace the string "None" (in case it was typed as text)
df["Region"] = df["Region"].replace("None", "-")


In [ ]:
df["Region"] = df["Region"].replace(r"^\s*$", "-", regex=True)


In [ ]:
df["Region"].value_counts(dropna=False)


In [ ]:
df.head(30)

In [ ]:
# Replace actual NaN/None
df["Gps Location"] = df["Gps Location"].fillna("-")

# Also handle empty strings or whitespace-only entries
df["Gps Location"] = df["Gps Location"].replace(r"^\s*$", "-", regex=True)


In [ ]:
df["Gps Location"].value_counts(dropna=False).head()


In [ ]:
df.head(30)

In [ ]:
df["Remarks"].unique()


In [ ]:
# Replace actual NaN values
df["Remarks"] = df["Remarks"].fillna("-")

# Replace strings that are just spaces or empty
df["Remarks"] = df["Remarks"].replace(r"^\s*$", "-", regex=True)


In [ ]:
df["Remarks"].value_counts(dropna=False).head(10)


In [ ]:
df = df.drop_duplicates()


In [ ]:
df.head(30)

In [ ]:
df = df.reset_index(drop=True)

df["Facility Id"] = df.index.map(lambda x: f"HF-{str(x).zfill(6)}")


In [ ]:
print("Is Facility Id unique?", df["Facility Id"].is_unique)


In [ ]:
df.head(30)

In [ ]:
# Ensure both columns are datetime just in case
df["Licence Issue Date"] = pd.to_datetime(df["Licence Issue Date"], dayfirst=True, errors='coerce')
df["Inspection Date"] = pd.to_datetime(df["Inspection Date"], dayfirst=True, errors='coerce')

# Find rows where inspection date is earlier than license issue date
mask = df["Inspection Date"] < df["Licence Issue Date"]

# Swap values where the mask is True
df.loc[mask, ["Licence Issue Date", "Inspection Date"]] = df.loc[mask, ["Inspection Date", "Licence Issue Date"]].values


In [ ]:
(df["Inspection Date"] < df["Licence Issue Date"]).any()


In [ ]:
df.head(30)

In [ ]:
df.to_csv("cleaned_health_registry.csv", index=False)


In [ ]:
import pandas as pd
import re

# Load the cleaned dataset
df = pd.read_csv("cleaned_health_registry.csv")

# ---------- 1. Check for missing values ----------
print("🧼 Missing Values Summary:")
print(df.isna().sum())
print("\n")

# ---------- 2. Validate Bed Capacity ----------
print("🚫 Invalid Bed Capacities (negative or non-integer):")
print(df[~df["Bed Capacity"].apply(lambda x: isinstance(x, (int, float)) and x >= 0)])
print("\n")

# ---------- 3. Validate Gps Location Format ----------
pattern = r"^-?\d+\.\d+,\s*-?\d+\.\d+$"
invalid_gps = df[~df["Gps Location"].astype(str).str.match(pattern)]
print("🌍 Invalid GPS Format Entries:")
print(invalid_gps)
print("\n")

# ---------- 4. Validate Categorical Fields ----------
print("📌 Unique Facility Types:")
print(df["Facility Type"].unique())
print("\n")

print("📌 Unique Remarks:")
print(df["Remarks"].unique())
print("\n")

print("📌 Unique Regions:")
print(df["Region"].unique())
print("\n")

# ---------- 5. Date Validation: Inspection must be >= Licence ----------
df["Licence Issue Date"] = pd.to_datetime(df["Licence Issue Date"], dayfirst=True, errors='coerce')
df["Inspection Date"] = pd.to_datetime(df["Inspection Date"], dayfirst=True, errors='coerce')

out_of_order = df[df["Inspection Date"] < df["Licence Issue Date"]]
print("📅 Entries Where Inspection Date Is Before Licence Issue Date:")
print(out_of_order)
print("\n")

# ---------- 6. Capitalization Standardization ----------
df["Facility Name"] = df["Facility Name"].str.title()
df["Region"] = df["Region"].str.title()

# ---------- 7. Whitespace Cleanup ----------
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# ---------- 8. Facility Id Uniqueness ----------
is_unique = df["Facility Id"].is_unique
print(f"🔍 Is Facility Id Unique? {is_unique}")
print("\n")

# ---------- Optional: Save validated version ----------
df.to_csv("validated_health_registry.csv", index=False)
print("✅ Cleaned and validated file saved as 'validated_health_registry.csv'")


In [ ]:
# Define keyword-based corrections
def correct_facility_type(row):
    name = str(row["Facility Name"]).lower()

    if "polyclinic" in name:
        return "Polyclinic"
    elif "clinic" in name:
        return "Polyclinic"  # unify all clinic types
    elif "health center" in name:
        return "Community Health Center"
    elif "center" in name:
        return "Community Health Center"
    elif "hospital" in name:
        return "Hospital"
    else:
        return row["Facility Type"]  # keep original if no clue

# Apply corrections
df["Facility Type"] = df.apply(correct_facility_type, axis=1)


In [ ]:
df.head(30)

In [ ]:
df["Facility Type"].unique()


In [ ]:
import pandas as pd

# 1. Load the cleaned data
df = pd.read_csv("cleaned_health_registry.csv")

# 2. Treat blanks and dashes as missing
df.replace(['', ' ', '-'], pd.NA, inplace=True)

# 3. Identify rows with any missing data
missing_data_df = df[df.isna().any(axis=1)]

# 4. Save rows with missing data
missing_data_df.to_csv("missing_data.csv", index=False)

# 5. Remove rows with missing data from the main DataFrame
df_cleaned = df.dropna()

# 6. Save the updated cleaned dataset
df_cleaned.to_csv("cleaned_health_registry.csv", index=False)

print(f"✅ {len(missing_data_df)} rows with missing data exported to 'missing_data.csv'.")
print(f"✅ Cleaned dataset saved with {len(df_cleaned)} records.")


In [ ]:
import pandas as pd

df = pd.read_csv("cleaned_health_registry.csv")
df.shape
df.isna().sum()


In [ ]:
df["Facility Type"].unique()
df["Region"].unique()


In [ ]:
(df == '-').sum()
(df == '').sum()


In [ ]:
# ensure  columns are in correct data type format
df["Licence Issue Date"] = pd.to_datetime(df["Licence Issue Date"], errors="coerce", dayfirst=True)
df["Inspection Date"] = pd.to_datetime(df["Inspection Date"], errors="coerce", dayfirst=True)
df["Facility Type"] = df["Facility Type"].astype("category")
df["Region"] = df["Region"].astype("category")
df["Remarks"] = df["Remarks"].astype("category")
df[['Latitude', 'Longitude']] = df['Gps Location'].str.split(', ', expand=True).astype(float)
df.info()


In [ ]:
df["Facility Id"] = df["Facility Id"].astype("string")
df["Facility Name"] = df["Facility Name"].astype("string")
df.drop(columns=["Gps Location"], inplace=True)
df.info()

In [ ]:
df.head(10)

In [ ]:
df.describe(include='all')  # Full summary


In [ ]:
%pip install ydata-profiling


In [ ]:
from ydata_profiling import ProfileReport

# Create the profile
profile = ProfileReport(df, title="Health Registry Dataset Profile", explorative=True)

# Save to HTML
profile.to_file("health_registry_profile.html")


In [ ]:
df.info()
